## Import Dependencies

In [1]:
from azureml.core import Workspace,Environment,Experiment,ScriptRunConfig
from azureml.core.compute import AmlCompute, ComputeTarget
from azureml.core.compute_target import ComputeTargetException

c:\Users\HP\anaconda3\lib\site-packages\paramiko\transport.py:219: CryptographyDeprecationWarning: Blowfish has been deprecated
  "class": algorithms.Blowfish,


In [2]:
resource_name = "pysdk"
workspace_name = "mlwspysdk"
subscriptionID = "2152fd0e-4534-43c6-9496-6e6de93b53db" 
aml_compute_target = "cluster" #ALL SMALL LETTER, NO underscore, 16ws long only.
experiment_name= 'expirement1'

## Now create Workspace


In [ ]:
try:
    ws=Workspace.from_config()
    print('Workspace is already exist')
except:
    ws=Workspace.create(workspace_name, 
                    resource_group=resource_name,
                    create_resource_group=True,
                    subscription_id=subscriptionID,
                    location="East US")
    ws.write_config('.azureml')


## Create Compute Target


In [ ]:
try:
    aml_compute = AmlCompute(ws, aml_compute_target)
    print("This Compute Target already exist.")
except ComputeTargetException:
    print("creating new compute target :",aml_compute_target)
    
    provisioning_config = AmlCompute.provisioning_configuration(vm_size = "STANDARD_D2_V2",
                                                                min_nodes = 1, 
                                                                max_nodes = 4,
                                               idle_seconds_before_scaledown=3000)    
    aml_compute = ComputeTarget.create(ws, aml_compute_target, provisioning_config)
    aml_compute.wait_for_completion(show_output=True, min_node_count=None, timeout_in_minutes=20)
    
print("Azure Machine Learning Compute attached now")


## Create Experiment


In [ ]:
exp = Experiment(ws,experiment_name)

## Create environment to execute your code


In [ ]:
env = Environment.from_conda_specification(name="azure_ml",file_path="./envfile.yml")
config=ScriptRunConfig(source_directory="./",script="mytrain.py",compute_target=aml_compute_target,environment=env)
execution=exp.submit(config)
execution.wait_for_completion(show_output=True)

## Register model in workspace


In [ ]:
from azureml.core import Model
model = Model.register(ws,model_path="./diabeticmodel.pkl",model_name="first_ml_model")

In [14]:
model = Model(ws,"first_ml_model")